In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
import quadratic_weighted_kappa
import feature_generator
from XgBoost import XGBoostModel
from CutPoints import CutPointOptimizer
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import pearsonr

In [2]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    

dfTrain['Product_Info_2_char'] = dfTrain["Product_Info_2"].str[0]
dfTrain['Product_Info_2_num'] = dfTrain["Product_Info_2"].str[1]

dfTest['Product_Info_2_char'] = dfTest["Product_Info_2"].str[0]
dfTest['Product_Info_2_num'] = dfTest["Product_Info_2"].str[1]

dfTrain.to_csv(path_or_buf='train.csv', index=False)
dfTest.to_csv(path_or_buf='test.csv', index=False)

IOError: File train.csv does not exist

In [ ]:
trainNanSum = dfTrain.isnull().sum(axis=1)
testNanSum = dfTest.isnull().sum(axis=1)

dfTrain['NanSum'] = trainNanSum
dfTest['NanSum'] = testNanSum

dfTrain.to_csv(path_or_buf='train.csv', index=False)
dfTest.to_csv(path_or_buf='test.csv', index=False)

In [ ]:
medKeyWordColumns = list()

for column in dfTrain.columns:
    if "Medical_Keyword" not in column:
        continue

    pearson = pearsonr(dfTrain[column], dfTrain.Response)
    if pearson[1] > 0.05:
        continue
    val = pearson[0]
    
    if val < 0:
        medKeyWordColumns.append(column)       
 
dfTrain['Good_Medical_Keyword_Total'] = dfTrain[medKeyWordColumns].sum(axis=1)
dfTest['Good_Medical_Keyword_Total'] = dfTest[medKeyWordColumns].sum(axis=1)

dfTrain.to_csv(path_or_buf='train.csv', index=False)
dfTest.to_csv(path_or_buf='test.csv', index=False)

In [ ]:
overallMean = np.mean(labels)
dfTrain['Med_History_2_Occ'] = np.NAN
dfTest['Med_History_2_Occ'] = np.NAN

for val in dfTrain["Medical_History_2"].unique():
    rowsWithVal = dfTrain[dfTrain["Medical_History_2"] == val]   
    dfTrain.loc[dfTrain["Medical_History_2"] == val, 'Med_History_2_Response'] = len(rowsWithVal)

for val in dfTest["Medical_History_2"].unique():
    rowsWithVal = dfTrain[dfTrain["Medical_History_2"] == val]
    dfTest.loc[dfTest["Medical_History_2"] == val, 'Med_History_2_Response'] = len(rowsWithVal)                      

In [ ]:
print quadratic_weighted_kappa.quadratic_weighted_kappa(dfTrain['Med_History_2_Response'], labels) 
print dfTest['Med_History_2_Response'].isnull().sum()
print dfTest['Med_History_2_Response'].notnull().sum()

In [ ]:
folds = 5
kf = KFold(len(train), folds)
qwks = list()

nearestNeighborsMissing = np.zeros(len(miss))

for train_index, test_index in kf:
    
    neigh = KNeighborsRegressor(n_neighbors=10, metric='manhattan', weights='distance')
    neigh.fit(dfTrain.iloc[train_index]['Med_History_2_Response'].values, labels.iloc[train_index])
    predictions = neigh.predict(dfTrain.iloc[test_index]['Med_History_2_Response'].values )
    
    qwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, labels.iloc[test_index])
    nearestNeighborsMissing[test_index] = predictions
    print qwk
    qwks.append(qwk)
    
print "mean %s" % str(np.mean(qwks))
print "median %s" % str(np.median(qwks))

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=10, metric='manhattan', weights='distance')
neigh.fit(miss, labels)
testNeighborPredictions = neigh.predict(test[missingDataColumns])